In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import io
import os
import re
import string
import shutil
from tqdm.notebook  import tqdm
import pickle
from IPython.display import clear_output
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Lambda, BatchNormalization
from tensorflow.keras.layers import TextVectorization

import keras.backend as K
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras.utils import np_utils

gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)


%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [40]:
def clean(doc):
    r_string = set(doc) - set(['A', 'B', 'C', 'D', 'E', 'F', 'G', 
                                    'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 
                                    'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                                    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 
                                    'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
                                    's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ',
                                      '1','2','3','4','5','6','7','8','9', '0','Á', 'Å', 'Æ', 'É',
                                  'Í', 'Î', 'Ö', '×', 'Ø', 'Ú', 'Ü', 'Þ', 'à', 'á', 'â', 'ã', 'ä', 'å', 
                                  'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ú',
                                  'û', 'ü', 'Ā', 'ā', 'ă', 'ć', 'č', 'Đ', 'đ', 'ė', 'ī', 'Ł', 'ł', 'ń', 'Ō', 'ō',
                                  'ś', 'ş', 'š', 'ū', 'ų', 'Ż',
                                  'ž', 'ơ', 'ư',
                                  'ḥ', 'ṃ', 'ṅ', 'ṣ', 'ṭ', 'ṯ', 'ả', 'ấ', 'ầ', 'ắ', 'ễ', 'ệ', 'ị', 'ớ', 'ử', 'ỳ','½','.',
                            
                                 ])
    r_list = sorted(list(r_string))
    print('Removed  --' + ''.join(r_list) +  '\n')
    document_ = doc

    for r in r_list:
        document_ =  document_.replace(r,"")

    return document_


In [50]:
corpus = tf.io.read_file('./wiki.train.tokens')
corpus =tf.strings.lower(corpus)
corpus = corpus.numpy()
corpus = corpus.decode('utf-8')
corpus = clean(corpus)
one_doc = [corpus]
corpus =  corpus.split('.')
corpus = [line for line in corpus if line is not ' ']

Removed  --
!"#$%&'()*+,-/:;<=>?@[\]^`|~¡£¥§°±²³µ·ʻʿ̃αβγκμСавекостяاحصلنه्กงณตมยรลัาิ่์გდვზიკორსუცძწხჯ჻‑–—‘’“”„†…′″⁄₤€₹⅓⅔→−≤☉♭♯〈〉のァアキスットプュリルヴ・動場大戦攻機殻火礮空隊﻿～

